In [14]:
def payoff(history):
    a, b = history
    if a == b:
        return 0
    elif a == 0:
        if b == 1:
            return -1
        return 2
    elif a == 1:
        if b == 2:
            return -2
        return 1
    else:
        if b == 0:
            return -2
        return 2


def matching(regrets):
    rplus = [max(0, r) for r in regrets]
    denom = sum(rplus)

    if denom > 0:
        return [r / denom for r in rplus]
    return [1 / 3, 1 / 3, 1 / 3]


def walk(history, i, p0, p1, R, S, t=1):
    if len(history) == 2:
        off = payoff(history)
        if i == 0:
            return off
        return -off

    player = len(history)
    strat = matching(R)
    cf = [0] * 3
    value = 0

    for action, prob in enumerate(strat):
        if player == 0:
            cf[action] = walk(history + [action], i, p0 * prob, p1, R, S)
        else:
            cf[action] = walk(history + [action], i, p0, p1 * prob, R, S)
        value += prob * cf[action]
    
    pi, pmi = p0, p1
    if player == 1:
        pi, pmi = p1, p0

    if player == i:
        for action, prob in enumerate(strat):
            R[action] += pmi * (cf[action] - value)
            S[action] += t * pi * prob
    
    return value


def cfr(maxiter):
    R = [[0] * 3 for _ in range(2)]
    S = [[0] * 3 for _ in range(2)]

    for t in range(1, 1 + maxiter):
        for player in range(2):
            walk([], player, 1, 1, R[player], S[player], t)

    return normalize(S[0]), normalize(S[1])


def normalize(v):
    s = sum(v)
    return [x / s for x in v]

In [15]:
cfr(10000)

([0.40912823922060165, 0.3888230107713464, 0.202048750008052],
 [0.4068718692022249, 0.39360712199917164, 0.19952100879860346])